# XGBoost

## Importing the libraries

In [21]:
import pandas as pd

## Importing the dataset

In [22]:
dataset = pd.read_csv("./filez/Data.csv")
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values
dataset.head()

,Sample code number,Clump Thickness,Uniformity of Cell Size,Uniformity of Cell Shape,Marginal Adhesion,Single Epithelial Cell Size,Bare Nuclei,Bland Chromatin,Normal Nucleoli,Mitoses,Class
0,1000025,5,1,1,1,2,1,3,1,1,2
1,1002945,5,4,4,5,7,10,3,2,1,2
2,1015425,3,1,1,1,2,2,3,1,1,2
3,1016277,6,8,8,1,3,4,3,7,1,2
4,1017023,4,1,1,3,2,1,3,1,1,2


## Splitting the dataset into the Training set and Test set

In [23]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

## Training XGBoost on the Training set

In [24]:
# @dev: The XGBoost classifier by default expects binary class labels to be [0, 1],
#       but in our dataset, the unique values are [2, 4], so we need to recode y to 0 and 1
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
 # This will transform 2, 4 to 0, 1
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.fit_transform(y_test)

In [25]:
from xgboost import XGBClassifier

classifier = XGBClassifier()
classifier.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

## Making the Confusion Matrix

In [26]:
from sklearn.metrics import confusion_matrix, accuracy_score

y_pred = classifier.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[85  2]
 [ 1 49]]


0.9781021897810219

## Applying k-Fold Cross Validation

In [27]:
from sklearn.model_selection import cross_val_score

accuracies = cross_val_score(estimator=classifier, X=X_train, y=y_train, cv=10)
print("Accuracy: {:.2f} %".format(accuracies.mean() * 100))
print("Standard Deviation: {:.2f} %".format(accuracies.std() * 100))

Accuracy: 96.71 %
Standard Deviation: 2.28 %


### Final Ranking

**$R^2$ score** results (ordered by better performance):

| Method    | $R^2$ score |
|--------------------------|--------------------------------------------------|
| XG Boost    | 0.9671  |
| Random Forest Regression    | 0.9615  |
| Support Vector Regression    | 0.9480  |
| Polynomial Regression (n=4)   | 0.9458  |
| Multiple Linear Regression    | 0.9325  |
| Decision Tree Regression    | 0.9229  |

*The non XG-Boost methods are not k-fold validated, but just over one sample, so probably should be lower.*